In [1]:
#
# En este dataset se desea pronosticar el precio de vehículos usados. El dataset
# original contiene las siguientes columnas:
#
# - Car_Name: Nombre del vehiculo.
# - Year: Año de fabricación.
# - Selling_Price: Precio de venta.
# - Present_Price: Precio actual.
# - Driven_Kms: Kilometraje recorrido.
# - Fuel_type: Tipo de combustible.
# - Selling_Type: Tipo de vendedor.
# - Transmission: Tipo de transmisión.
# - Owner: Número de propietarios.
#
# El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
# en la carpeta "files/input/".
#
# Los pasos que debe seguir para la construcción de un modelo de
# pronostico están descritos a continuación.
#
#
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.
#

# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle
import gzip
import os
import json
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
import zipfile

# Cargue de los datos
train_path = "../files/input/train_data.csv.zip"
test_path = "../files/input/test_data.csv.zip"
extract_path = "../files/input/"

# Ahora, extraigamos la data del dataset 
if not os.path.exists("../files/input/train_data.csv"):
  with zipfile.ZipFile(train_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

if not os.path.exists("../files/input/test_data.csv"):
  with zipfile.ZipFile(test_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Ahora, leamos la data
train = pd.read_csv("../files/input/train_data.csv")
test = pd.read_csv("../files/input/test_data.csv")

# Ahora, el preprocesamiento

# Crear columna Age en train y test
train['Age'] = 2021 - train['Year']
test['Age'] = 2021 - test['Year']

# Elimine las columnas 'Year' y 'Car_Name'
train_clean = train.drop(columns=['Year','Car_Name'])
test_clean = test.drop(columns=['Year','Car_Name'])

train.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,jazz,2016,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,i10,2013,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,TVS Apache RTR 180,2011,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,eon,2016,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,Royal Enfield Thunder 350,2013,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [2]:
#
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.


X_train = train_clean.drop(columns="Present_Price")
y_train = train_clean["Present_Price"]
X_test = test_clean.drop(columns="Present_Price")
y_test = test_clean["Present_Price"]

In [3]:
#
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

categorical_features = ['Fuel_Type','Selling_type','Transmission']
numeric_features = [col for col in X_train.columns if col not in categorical_features]

# Preprocesamiento para las variables categóricas
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Preprocesamiento para las variables numéricas
numerical_transformer = MinMaxScaler()

# Combinación de preprocesadores

preprocessor = ColumnTransformer(
    transformers = [
        ["encoder", categorical_transformer, categorical_features],
        ["numerica", numerical_transformer, numeric_features]
    ]
)

# Creación del pipeline
pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("feature_selection", SelectKBest(score_func=f_regression)),
        ("classifier", LinearRegression()),
    ]
)

In [4]:
# Paso 4.
# Optimice los hiperparámetros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.

from sklearn.model_selection import GridSearchCV

param_grid = {
    'feature_selection__k': range(1, 12),
    'classifier__fit_intercept': [True, False],
    'classifier__positive': [True, False]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    refit=True,
    verbose=1
)

grid_search.fit(X_train, y_train)

# Mejor scoring (recuerda: es negativo)
print("Mejor neg_mean_absolute_error (CV):", grid_search.best_score_)

# Convertir a MAE positivo para interpretación
print("Mejor MAE (CV):", -grid_search.best_score_)

# Obtener el mejor modelo ya entrenado
best_model = grid_search.best_estimator_
print("Modelo seleccionado:", best_model)



Fitting 10 folds for each of 44 candidates, totalling 440 fits
Mejor neg_mean_absolute_error (CV): -1.765768769272933
Mejor MAE (CV): 1.765768769272933
Modelo seleccionado: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[['encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Fuel_Type', 'Selling_type',
                                                   'Transmission']],
                                                 ['numerica', MinMaxScaler(),
                                                  ['Selling_Price',
                                                   'Driven_kms', 'Owner',
                                                   'Age']]])),
                ('feature_selection',
                 SelectKBest(k=11,
                             score_func=<function f_regression at 0x000002BAE54724D0>)),
                ('classifier', LinearRegression(p

In [5]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usando la libreria gzip.

# Paso 5 — Guardar el modelo entrenado como files/models/model.pkl.gz
import gzip, pickle
from pathlib import Path

MODEL_FILENAME = Path("../files/models/model.pkl.gz")
MODEL_FILENAME.parent.mkdir(parents=True, exist_ok=True)

# ¡IMPORTANTE! guarda el GridSearchCV completo (no solo best_estimator_)
with gzip.open(MODEL_FILENAME, "wb") as f:
    pickle.dump(grid_search, f)

# (Opcional) Verificar que se puede cargar en el MISMO entorno/versión
with gzip.open(MODEL_FILENAME, "rb") as f:
    _ = pickle.load(f)


In [6]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guárdelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error
import os, json

from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error
import os, json, gzip, pickle

# Usemos el mejor modelo
best_model = grid_search.best_estimator_

# Predicciones
y_train_pred = best_model.predict(X_train)
y_test_pred  = best_model.predict(X_test)

# Crear carpeta de salida si no existe
os.makedirs("../files/output", exist_ok=True)

# Métricas para train y test
train_metrics = {
    "type": "metrics",
    "dataset": "train",
    "r2": r2_score(y_train, y_train_pred),
    "mse": mean_squared_error(y_train, y_train_pred),
    "mad": median_absolute_error(y_train, y_train_pred)
}

test_metrics = {
    "type": "metrics",
    "dataset": "test",
    "r2": r2_score(y_test, y_test_pred),
    "mse": mean_squared_error(y_test, y_test_pred),
    "mad": median_absolute_error(y_test, y_test_pred)
}

# Guardar en JSON con una línea por diccionario
with open("../files/output/metrics.json", "w") as f:
    f.write(json.dumps(train_metrics) + "\n")
    f.write(json.dumps(test_metrics) + "\n")

print("Métricas guardadas en ../files/output/metrics.json")
print(train_metrics)
print(test_metrics)


Métricas guardadas en ../files/output/metrics.json
{'type': 'metrics', 'dataset': 'train', 'r2': 0.8916962358587399, 'mse': 5.8746462805982045, 'mad': 1.0929123440195472}
{'type': 'metrics', 'dataset': 'test', 'r2': 0.7325716754123305, 'mse': 32.566672753866264, 'mad': 1.5033540603205724}
